In [ ]:
%matplotlib inline

%load_ext autoreload
%autoreload 2

## Non-std libs
from matplotlib.colors import hsv_to_rgb
import matplotlib.pyplot as plt

## Local modules
from scn_rrd import rrd_meta_utils, rrd_utils, plot_utils

### Count users by counting UEs

In [ ]:
class At_eNB:
    def __init__(self):
        self.locations = [
            'FCS',
            'sps-franklin',
        ]
        
        meta = rrd_meta_utils.read_meta()
        self.meta = meta[
            ( meta['location'].isin(self.locations) ) &
            ( meta['device_group_name'] == 'eNB' )
        ]
        devices_info = self.meta[['hostname', 'location']].drop_duplicates()
        self.hostname_to_location = {
            hostname : location
            for (_, (hostname, location)) in devices_info.iterrows()
        }
    
    def read_rrds(self):
        self.hostname_to_df = rrd_utils.read_rrds(
            self.hostname_to_location.keys(),
            rrd_meta_utils.ENB_DEVICE_RRD_FILENAMES.clients,
            '-2month'
        )
        for df in self.hostname_to_df.values():
            df.rename(columns={'sensor': 'users_ct'}, inplace=True)

In [ ]:
at_enb = At_eNB()

In [ ]:
at_enb.read_rrds()
# Right now, BaicellsNova436Q devices do not have wireless-sensor-clients-rts-1.rrd.
# If we see stderr here, this is expected.

### Count users by DHCP

In [ ]:
class At_DHCP:
    def __init__(self):
        self.locations = [
            'lihi-southend',
            'nickelsville-cd',
            'northlake',
        ]
        
        meta = rrd_meta_utils.read_meta()
        self.meta = meta[
            ( meta['location'].isin(self.locations) ) &
            ( meta['device_group_name'].isin(['CPE gateway', 'Backhaul gateway']) )
        ]
        devices_info = self.meta[['hostname', 'location']].drop_duplicates()
        self.hostname_to_location = {
            hostname : location
            for (_, (hostname, location)) in devices_info.iterrows()
        }
    
    def read_rrds(self):
        self.hostname_to_df = rrd_utils.read_rrds(
            self.hostname_to_location.keys(),
            rrd_meta_utils.MIKROTIK_DEVICE_RRD_FILENAMES.clients,
            '-2month'
        )
        for df in self.hostname_to_df.values():
            df.rename(columns={'leases': 'users_ct'}, inplace=True)

In [ ]:
at_dhcp = At_DHCP()

In [ ]:
at_dhcp.read_rrds()

### Plot

In [ ]:
(fig, ax) = plt.subplots(figsize=(12, 6))
for namespace in [at_enb, at_dhcp]:
    for (hostname, df) in namespace.hostname_to_df.items():
        location = namespace.hostname_to_location[hostname]
        color_h = plot_utils.LOC_TO_COLOR_H[location]
        ax.plot(df['time'], df['users_ct'], label=hostname, color=hsv_to_rgb((color_h, 1, 1)))
ax.set_title('Count of users')
ax.tick_params(axis='x', labelrotation=90)
ax.legend()
None # Hide stdout output of the above line